# 自動でBBO-Rietvledを実行

In [9]:
# import packages
# %matplotlib inline

from IPython.display import clear_output

import subprocess
import os
import sys
import glob
from multiprocessing import Process, Queue
import pandas as pd
import optuna
# import matplotlib.pyplot as plt
# from matplotlib.gridspec import GridSpec
import time
import numpy as np
sys.path.append('/opt/conda/GSASII/')
import re
import datetime
import GSASIIscriptable as G2sc

DATABASE_DIR = "/bbo_rietveld/database"


In [10]:
def tio2_bbor_run(target_file_path):
    target_file_name = os.path.basename(target_file_path)
    class ProjectTiO2:
        def __init__(self, trial_number):
            self.gpx = G2sc.G2Project(newgpx=os.path.join(WORK_DIR, "trial{}_{}.gpx".format(trial_number,newname)))
            
            G2sc.SetPrintLevel('none')

            self.hist1 = self.gpx.add_powder_histogram(
                target_file_path, os.path.join(DATABASE_DIR, 'Lab_CuKa.instprm'), fmthint='xye')

            self.phase0 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'TiO2_anatase.cif'),
                                            phasename='anatase',
                                            histograms=[self.hist1])
            self.phase1 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'TiO2_rutile.cif'),
                                            phasename='rutile',
                                            histograms=[self.hist1])            
            
            self.hist1.data['Sample Parameters']['Scale'][1] = False
            self.gpx.data['Controls']['data']['max cyc'] = 10
            self.phase0.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True
            self.phase1.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True

            # Set to use iso
            for val in self.phase0.data['Atoms']:
                val[9] = 'I'
            for val in self.phase1.data['Atoms']:
                val[9] = 'I'

        def refine_and_calc_Rwp(self, param_dict):
            self.gpx.do_refinements(refinements=[param_dict])
            for hist in self.gpx.histograms():
                _, Rwp = hist.name, hist.get_wR()
            return Rwp

    # objective function

    def objective(trial):
        ### define search space ###
        sample_parameters0_refine =[]
        for p in ['phase0']:
            if not trial.suggest_categorical('Phase_parameters refine %s' % (p), [True, False]):
                sample_parameters0_refine.append(p)
                #     使わない相をここで消す
        not_ref_phases = sample_parameters0_refine
        refdict0 = {'set': {'Limits': [twot_srart, twot_end]}, 'refine': True}

        # Background
        background_type = trial.suggest_categorical(
            'Background type', ['chebyschev',
                                'chebyschev-1',
                                'cosine',
                                'Q^2 power series',
                                'Q^-2 power series'
                               ])
        no_coeffs = trial.suggest_int('Number of coefficients', 10, 15 + 1)  # [1, 16)
        background_refine = trial.suggest_categorical('Background refine', [True, False])
        refdict0bg_h = {
            'set': {
                'Background': {
                    'type': background_type,
                    'no. coeffs': no_coeffs,
                    'refine': background_refine
                }
            }
        }

        # Instrument parameters
        instrument_parameters1_refine = []
        for p in ['Zero']:
            if trial.suggest_categorical('Instrument_parameters refine %s' % (p), [True, False]):
                instrument_parameters1_refine.append(p)
        refdict1_h = {'set': {'Cell': True, 'Instrument Parameters': instrument_parameters1_refine}}

        
        sample_parameters1_refine =[]
        for p in ['Shift', 'SurfRoughA','SurfRoughB']:
            if trial.suggest_categorical('Sample_parameters refine %s' % (p), [True, False]):
                sample_parameters1_refine.append(p)
        refdict1_h2 = {"set": {'Sample Parameters':sample_parameters1_refine }}

        instrument_parameters2_refine = []
        for p in ['U', 'V', 'W', 'X', 'Y', 'SH/L']:
            if trial.suggest_categorical('Peakshape_parameters refine %s' % (p), [True, False]):
                instrument_parameters2_refine.append(p)
        refdict2_h = {'set': {'Instrument Parameters': instrument_parameters2_refine}}

        refdict3_h_2 = {'set': {'Atoms': {'all': 'XU'}}}
        
        
        ### add new params ### yotsu
        mustrain_refine = trial.suggest_categorical('Mustrain refine', [True, False])
        refdict_new  ={
            'set':{
                'Mustrain':{
                    'type':'uniaxial',
                    'direction': [0, 0, 1],
                    'refine': mustrain_refine
                }
            }
        }
        
        
        # Limits (wide angle)
        refdict_fin_h = {'set': {'Limits': [twot_srart, twot_end]}, 'refine': True}
        

        # Evaluate
        refine_params_list = [
            not_ref_phases,
            refdict0,
            refdict0bg_h,
            refdict1_h,
            refdict1_h2,
            refdict2_h,
            refdict3_h_2,
            refdict_fin_h,
            refdict_new
                             ]

        def evaluate(trial_number, refine_params_list, q):
            ERROR_PENALTY = 1e9
            try:
                project = ProjectTiO2(trial_number)
                for leng in range(len(project.gpx.phases())):
                    project.gpx.phases()[leng].data['pId'] = leng
                for params in refine_params_list[1:]:
                    Rwp = project.refine_and_calc_Rwp(params)
                for i in range(len(project.gpx.phases())):
                    phase_TiO2 = project.gpx.phases()[i]
                    u_iso_list = [atom.uiso for atom in phase_TiO2.atoms()]
                scale_list = [i['Histograms']['PWDR '+target_file_name]['Scale'][0] for i in project.gpx.phases()]
                if min(scale_list) < 0.001:
                    Rwp = ERROR_PENALTY
                q.put(Rwp)

            except Exception as e:
                q.put(ERROR_PENALTY)

        q = Queue()
        p = Process(target=evaluate, args=(trial.number, refine_params_list, q))
        p.start()
        Rwp = q.get()
        p.join()

        return Rwp

    # import datetime
    start_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
    newname = "{}".format(target_file_name.strip(".csv"))
    study = optuna.create_study(study_name=newname
                            ,sampler=optuna.samplers.TPESampler(n_startup_trials=random_trials, seed=RANDOM_SEED))#, multivariate=True))
    start = time.time()
    study.optimize(objective, n_trials=trials, n_jobs=jobs)
    end = time.time()-start
    df = study.trials_dataframe()
    df.columns = [' '.join(col).replace('params', '').strip().replace(' ','') for col in df.columns.values]
    df.rename(columns={'value':'Rwp', 'number':'trial'}, inplace=True)
    df.to_pickle(os.path.join(WORK_DIR,newname+".pkl"))

    best_trial_number = study.best_trial.number
    best_rwp = int(study.best_trial.value)

    new_dir_name = newname + start_time.strftime('_%Y%m%d_%H%M%S') + "_trial{}".format(best_trial_number) + "_Rwp{}".format(best_rwp)
    newdir = os.path.join(WORK_DIR, new_dir_name)

    # result directory
    result_path = os.path.join(RESULT_DIR, new_dir_name)
    subprocess.run("mkdir {}".format(result_path), shell=True)

    df.to_csv(result_path + "/result.csv", mode='w')
    subprocess.run("cp {} {}".format(target_file_path, result_path), shell=True)
    subprocess.run("cp {} {}".format(os.path.join(WORK_DIR,"trial{}_{}.lst".format(study.best_trial.number,newname)), result_path), shell=True)
    subprocess.run("cp {} {}".format(os.path.join(WORK_DIR,"trial{}_{}.gpx".format(study.best_trial.number,newname)), result_path), shell=True)
    subprocess.run("cp {}/*.pkl {}".format(WORK_DIR,result_path), shell=True)

    # work directory
    subprocess.run("mkdir {}".format(newdir), shell=True)
    subprocess.run("mv {}/*.gpx {}".format(WORK_DIR,newdir), shell=True)
    subprocess.run("mv {}/*.lst {}".format(WORK_DIR,newdir), shell=True)
    subprocess.run("mv {}/*.pkl {}".format(WORK_DIR,newdir), shell=True)

    # output data for graph
    data_path = os.path.join(result_path,"trial{}_{}.gpx".format(study.best_trial.number,newname))
    gpx = G2sc.G2Project(data_path)
    for i,h in enumerate(gpx.histograms()):
        if i > 0:
            raise Exception ('Multiple histograms is not applicable')
        hfil = os.path.splitext(data_path)[0]
        if os.path.exists(hfil+'.csv'):
            print ('{}.csv is already existed. No need to run this'.format(hfil))
            return
        print(h.name, "analyzed"+hfil+'.csv')
        h.Export(hfil,'.csv','histogram CSV')

    clear_output(True)

    print("Finished!!")
    print("Best trial:" + str(study.best_trial.number))
    print("Best Rwp:" + str(study.best_trial.value))
    print("Time: {:.2f} sec".format(end), " ({})".format(datetime.timedelta(seconds=end)))

In [11]:
def tio2_brookite_bbor_run(target_file_path):
    target_file_name = os.path.basename(target_file_path)
    class ProjectTiO2:
        def __init__(self, trial_number):
            self.gpx = G2sc.G2Project(newgpx=os.path.join(WORK_DIR, "trial{}_{}.gpx".format(trial_number,newname)))
            
            G2sc.SetPrintLevel('none')

            self.hist1 = self.gpx.add_powder_histogram(
                target_file_path, os.path.join(DATABASE_DIR, 'Lab_CuKa.instprm'), fmthint='xye')

            self.phase0 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'TiO2_anatase.cif'),
                                            phasename='anatase',
                                            histograms=[self.hist1])
            self.phase1 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'TiO2_rutile.cif'),
                                            phasename='rutile',
                                            histograms=[self.hist1])
            self.phase2 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'TiO2_brookite.cif'),
                                            phasename='brookite',
                                            histograms=[self.hist1])

            self.hist1.data['Sample Parameters']['Scale'][1] = False
            self.gpx.data['Controls']['data']['max cyc'] = 10
            self.phase0.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True
            self.phase1.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True
            self.phase2.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True

            # Set to use iso
            for val in self.phase0.data['Atoms']:
                val[9] = 'I'
            for val in self.phase1.data['Atoms']:
                val[9] = 'I'
            for val in self.phase2.data['Atoms']:
                val[9] = 'I'

        def refine_and_calc_Rwp(self, param_dict):
            self.gpx.do_refinements(refinements=[param_dict])
            for hist in self.gpx.histograms():
                _, Rwp = hist.name, hist.get_wR()
            return Rwp

    # objective function

    def objective(trial):
        ### define search space ###
        sample_parameters0_refine =[]
        for p in ['phase0']:
            if not trial.suggest_categorical('Phase_parameters refine %s' % (p), [True, False]):
                sample_parameters0_refine.append(p)
                #     使わない相をここで消す
        not_ref_phases = sample_parameters0_refine
        refdict0 = {'set': {'Limits': [twot_srart, twot_end]}, 'refine': True}

        # Background
        background_type = trial.suggest_categorical(
            'Background type', ['chebyschev',
                                'chebyschev-1',
                                'cosine',
                                'Q^2 power series',
                                'Q^-2 power series'
                               ])
        no_coeffs = trial.suggest_int('Number of coefficients', 10, 15 + 1)  # [1, 16)
        background_refine = trial.suggest_categorical('Background refine', [True, False])
        refdict0bg_h = {
            'set': {
                'Background': {
                    'type': background_type,
                    'no. coeffs': no_coeffs,
                    'refine': background_refine
                }
            }
        }

        # Instrument parameters
        instrument_parameters1_refine = []
        for p in ['Zero']:
            if trial.suggest_categorical('Instrument_parameters refine %s' % (p), [True, False]):
                instrument_parameters1_refine.append(p)
        refdict1_h = {'set': {'Cell': True, 'Instrument Parameters': instrument_parameters1_refine}}

        
        sample_parameters1_refine =[]
        for p in ['Shift', 'SurfRoughA','SurfRoughB']:
            if trial.suggest_categorical('Sample_parameters refine %s' % (p), [True, False]):
                sample_parameters1_refine.append(p)
        refdict1_h2 = {"set": {'Sample Parameters':sample_parameters1_refine }}

        instrument_parameters2_refine = []
        for p in ['U', 'V', 'W', 'X', 'Y', 'SH/L']:
            if trial.suggest_categorical('Peakshape_parameters refine %s' % (p), [True, False]):
                instrument_parameters2_refine.append(p)
        refdict2_h = {'set': {'Instrument Parameters': instrument_parameters2_refine}}

        refdict3_h_2 = {'set': {'Atoms': {'all': 'XU'}}}
        
        
        ### add new params ### yotsu
        mustrain_refine = trial.suggest_categorical('Mustrain refine', [True, False])
        refdict_new  ={
            'set':{
                'Mustrain':{
                    'type':'uniaxial',
                    'direction': [0, 0, 1],
                    'refine': mustrain_refine
                }
            }
        }
        
        
        # Limits (wide angle)
        refdict_fin_h = {'set': {'Limits': [twot_srart, twot_end]}, 'refine': True}
        

        # Evaluate
        refine_params_list = [
            not_ref_phases,
            refdict0,
            refdict0bg_h,
            refdict1_h,
            refdict1_h2,
            refdict2_h,
            refdict3_h_2,
            refdict_fin_h,
            refdict_new
                             ]

        def evaluate(trial_number, refine_params_list, q):
            ERROR_PENALTY = 1e9
            try:
                project = ProjectTiO2(trial_number)
                for leng in range(len(project.gpx.phases())):
                    project.gpx.phases()[leng].data['pId'] = leng
                for params in refine_params_list[1:]:
                    Rwp = project.refine_and_calc_Rwp(params)
                for i in range(len(project.gpx.phases())):
                    phase_TiO2 = project.gpx.phases()[i]
                    u_iso_list = [atom.uiso for atom in phase_TiO2.atoms()]
                scale_list = [i['Histograms']['PWDR '+target_file_name]['Scale'][0] for i in project.gpx.phases()]
                if min(scale_list) < 0.001:
                    Rwp = ERROR_PENALTY
                q.put(Rwp)

            except Exception as e:
                q.put(ERROR_PENALTY)

        q = Queue()
        p = Process(target=evaluate, args=(trial.number, refine_params_list, q))
        p.start()
        Rwp = q.get()
        p.join()

        return Rwp

    # import datetime
    start_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
    newname = "{}".format(target_file_name.strip(".csv"))
    study = optuna.create_study(study_name=newname
                            ,sampler=optuna.samplers.TPESampler(n_startup_trials=random_trials, seed=RANDOM_SEED))#, multivariate=True))
    start = time.time()
    study.optimize(objective, n_trials=trials, n_jobs=jobs)
    end = time.time()-start
    df = study.trials_dataframe()
    df.columns = [' '.join(col).replace('params', '').strip().replace(' ','') for col in df.columns.values]
    df.rename(columns={'value':'Rwp', 'number':'trial'}, inplace=True)
    df.to_pickle(os.path.join(WORK_DIR,newname+".pkl"))

    best_trial_number = study.best_trial.number
    best_rwp = int(study.best_trial.value)

    new_dir_name = newname + start_time.strftime('_%Y%m%d_%H%M%S') + "_trial{}".format(best_trial_number) + "_Rwp{}".format(best_rwp)
    newdir = os.path.join(WORK_DIR, new_dir_name)

    # result directory
    result_path = os.path.join(RESULT_DIR, new_dir_name)
    subprocess.run("mkdir {}".format(result_path), shell=True)

    df.to_csv(result_path + "/result.csv", mode='w')
    subprocess.run("cp {} {}".format(target_file_path, result_path), shell=True)
    subprocess.run("cp {} {}".format(os.path.join(WORK_DIR,"trial{}_{}.lst".format(study.best_trial.number,newname)), result_path), shell=True)
    subprocess.run("cp {} {}".format(os.path.join(WORK_DIR,"trial{}_{}.gpx".format(study.best_trial.number,newname)), result_path), shell=True)
    subprocess.run("cp {}/*.pkl {}".format(WORK_DIR,result_path), shell=True)

    # work directory
    subprocess.run("mkdir {}".format(newdir), shell=True)
    subprocess.run("mv {}/*.gpx {}".format(WORK_DIR,newdir), shell=True)
    subprocess.run("mv {}/*.lst {}".format(WORK_DIR,newdir), shell=True)
    subprocess.run("mv {}/*.pkl {}".format(WORK_DIR,newdir), shell=True)

    # output data for graph
    data_path = os.path.join(result_path,"trial{}_{}.gpx".format(study.best_trial.number,newname))
    gpx = G2sc.G2Project(data_path)
    for i,h in enumerate(gpx.histograms()):
        if i > 0:
            raise Exception ('Multiple histograms is not applicable')
        hfil = os.path.splitext(data_path)[0]
        if os.path.exists(hfil+'.csv'):
            print ('{}.csv is already existed. No need to run this'.format(hfil))
            return
        print(h.name, "analyzed"+hfil+'.csv')
        h.Export(hfil,'.csv','histogram CSV')

    clear_output(True)

    print("Finished!!")
    print("Best trial:" + str(study.best_trial.number))
    print("Best Rwp:" + str(study.best_trial.value))
    print("Time: {:.2f} sec".format(end), " ({})".format(datetime.timedelta(seconds=end)))

In [12]:
def mechano_bbor_run(target_file_path):
    target_file_name = os.path.basename(target_file_path)
    class ProjectMechano:
        def __init__(self, trial_number):
            self.gpx = G2sc.G2Project(newgpx=os.path.join(WORK_DIR, "trial{}_{}.gpx".format(trial_number,newname)))
            
            G2sc.SetPrintLevel('none')

            self.hist1 = self.gpx.add_powder_histogram(
                target_file_path, os.path.join(DATABASE_DIR, 'Lab_CuKa.instprm'), fmthint='xye')

            self.phase0 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'CsBr.cif'),
                                            phasename='CsBr',
                                            histograms=[self.hist1])
            self.phase1 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'PbBr2.cif'),
                                            phasename='PbBr2',
                                            histograms=[self.hist1])
            self.phase2 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'CsPbBr3_Pbnm.cif'),
                                             phasename='CsPbBr3',
                                             histograms=[self.hist1])
            self.phase3 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'Cs4PbBr6_fit.cif'),
                                             phasename='Cs4PbBr6',
                                             histograms=[self.hist1])         
            
            self.hist1.data['Sample Parameters']['Scale'][1] = False
            self.gpx.data['Controls']['data']['max cyc'] = 10
            self.phase0.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True
            self.phase1.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True
            self.phase2.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True
            self.phase3.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True
            

            # Set to use iso
            for val in self.phase0.data['Atoms']:
                val[9] = 'I'
            for val in self.phase1.data['Atoms']:
                val[9] = 'I'
            for val in self.phase2.data['Atoms']:
                val[9] = 'I'
            for val in self.phase3.data['Atoms']:
                val[9] = 'I'

        def refine_and_calc_Rwp(self, param_dict):
            self.gpx.do_refinements(refinements=[param_dict])
            for hist in self.gpx.histograms():
                _, Rwp = hist.name, hist.get_wR()
            return Rwp

    # objective function

    def objective(trial):
        ### define search space ###
        sample_parameters0_refine =[]
        for p in ['phase0']:
            if not trial.suggest_categorical('Phase_parameters refine %s' % (p), [True, False]):
                sample_parameters0_refine.append(p)
                #     使わない相をここで消す
        not_ref_phases = sample_parameters0_refine
        refdict0 = {'set': {'Limits': [twot_srart, twot_end]}, 'refine': True}

        # Background
        background_type = trial.suggest_categorical(
            'Background type', ['chebyschev',
                                'cosine',
                                'Q^2 power series',
                                'Q^-2 power series',
                                # 'lin interpolate',
                                'inv interpolate',
                                'log interpolate'])
        no_coeffs = trial.suggest_int('Number of coefficients', 1, 15 + 1)  # [1, 16)
        background_refine = trial.suggest_categorical('Background refine', [True, False])
        refdict0bg_h = {
            'set': {
                'Background': {
                    'type': background_type,
                    'no. coeffs': no_coeffs,
                    'refine': background_refine
                }
            }
        }

        # Instrument parameters
        instrument_parameters1_refine = []
        for p in ['Zero']:
            if trial.suggest_categorical('Instrument_parameters refine %s' % (p), [True, False]):
                instrument_parameters1_refine.append(p)
        refdict1_h = {'set': {'Cell': True, 'Instrument Parameters': instrument_parameters1_refine}}

        
        sample_parameters1_refine =[]
        for p in ['Shift', 'SurfRoughA','SurfRoughB']:
            if trial.suggest_categorical('Sample_parameters refine %s' % (p), [True, False]):
                sample_parameters1_refine.append(p)
        refdict1_h2 = {"set": {'Sample Parameters':sample_parameters1_refine }}

        instrument_parameters2_refine = []
        for p in ['U', 'V', 'W', 'X', 'Y', 'SH/L']:
            if trial.suggest_categorical('Peakshape_parameters refine %s' % (p), [True, False]):
                instrument_parameters2_refine.append(p)
        refdict2_h = {'set': {'Instrument Parameters': instrument_parameters2_refine}}

        refdict3_h_2 = {'set': {'Atoms': {'all': 'XU'}}}
        
        
        ### add new params ### yotsu
        mustrain_refine = trial.suggest_categorical('Mustrain refine', [True, False])
        refdict_new  ={
            'set':{
                'Mustrain':{
                    'type':'uniaxial',
                    'direction': [0, 0, 1],
                    'refine': mustrain_refine
                }
            }
        }
        
        
        # Limits (wide angle)
        refdict_fin_h = {'set': {'Limits': [twot_srart, twot_end]}, 'refine': True}
        

        # Evaluate
        refine_params_list = [
            not_ref_phases,
            refdict0,
            refdict0bg_h,
            refdict1_h,
            refdict1_h2,
            refdict2_h,
            refdict3_h_2,
            refdict_fin_h,
            refdict_new
                             ]

        def evaluate(trial_number, refine_params_list, q):
            ERROR_PENALTY = 1e9
            try:
                project = ProjectMechano(trial_number)
                for leng in range(len(project.gpx.phases())):
                    project.gpx.phases()[leng].data['pId'] = leng
                for params in refine_params_list[1:]:
                    Rwp = project.refine_and_calc_Rwp(params)
                for i in range(len(project.gpx.phases())):
                    phase_TiO2 = project.gpx.phases()[i]
                    u_iso_list = [atom.uiso for atom in phase_TiO2.atoms()]
                scale_list = [i['Histograms']['PWDR '+target_file_name]['Scale'][0] for i in project.gpx.phases()]
                if min(scale_list) < 0.001:
                    Rwp = ERROR_PENALTY
                q.put(Rwp)

            except Exception as e:
                q.put(ERROR_PENALTY)

        q = Queue()
        p = Process(target=evaluate, args=(trial.number, refine_params_list, q))
        p.start()
        Rwp = q.get()
        p.join()

        return Rwp

    # import datetime
    start_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
    newname = "{}".format(target_file_name.strip(".csv"))
    study = optuna.create_study(study_name=newname
                            ,sampler=optuna.samplers.TPESampler(n_startup_trials=random_trials, seed=RANDOM_SEED))#, multivariate=True))
    start = time.time()
    study.optimize(objective, n_trials=trials, n_jobs=jobs)
    end = time.time()-start
    df = study.trials_dataframe()
    df.columns = [' '.join(col).replace('params', '').strip().replace(' ','') for col in df.columns.values]
    df.rename(columns={'value':'Rwp', 'number':'trial'}, inplace=True)
    df.to_pickle(os.path.join(WORK_DIR,newname+".pkl"))

    best_trial_number = study.best_trial.number
    best_rwp = int(study.best_trial.value)

    new_dir_name = newname + start_time.strftime('_%Y%m%d_%H%M%S') + "_trial{}".format(best_trial_number) + "_Rwp{}".format(best_rwp)
    newdir = os.path.join(WORK_DIR, new_dir_name)

    # result directory
    result_path = os.path.join(RESULT_DIR, new_dir_name)
    subprocess.run("mkdir {}".format(result_path), shell=True)

    df.to_csv(result_path + "/result.csv", mode='w')
    subprocess.run("cp {} {}".format(target_file_path, result_path), shell=True)
    subprocess.run("cp {} {}".format(os.path.join(WORK_DIR,"trial{}_{}.lst".format(study.best_trial.number,newname)), result_path), shell=True)
    subprocess.run("cp {} {}".format(os.path.join(WORK_DIR,"trial{}_{}.gpx".format(study.best_trial.number,newname)), result_path), shell=True)
    subprocess.run("cp {}/*.pkl {}".format(WORK_DIR,result_path), shell=True)

    # work directory
    subprocess.run("mkdir {}".format(newdir), shell=True)
    subprocess.run("mv {}/*.gpx {}".format(WORK_DIR,newdir), shell=True)
    subprocess.run("mv {}/*.lst {}".format(WORK_DIR,newdir), shell=True)
    subprocess.run("mv {}/*.pkl {}".format(WORK_DIR,newdir), shell=True)

    # output data for graph
    data_path = os.path.join(result_path,"trial{}_{}.gpx".format(study.best_trial.number,newname))
    gpx = G2sc.G2Project(data_path)
    for i,h in enumerate(gpx.histograms()):
        if i > 0:
            raise Exception ('Multiple histograms is not applicable')
        hfil = os.path.splitext(data_path)[0]
        if os.path.exists(hfil+'.csv'):
            print ('{}.csv is already existed. No need to run this'.format(hfil))
            return
        print(h.name, "analyzed"+hfil+'.csv')
        h.Export(hfil,'.csv','histogram CSV')

    clear_output(True)

    print("Finished!!")
    print("Best trial:" + str(study.best_trial.number))
    print("Best Rwp:" + str(study.best_trial.value))
    print("Time: {:.2f} sec".format(end), " ({})".format(datetime.timedelta(seconds=end)))

In [13]:
### SET YOUR PARAMETER ###
user = "kawasaki"
STUDY_NAME = 'KKYN6'
target_dir = "/bbo_rietveld/kawasaki/nas"

twot_srart = 10
twot_end = 60

RANDOM_SEED =  1024
random_trials=20
trials=200
jobs=20

now = datetime.datetime.now()
today = now.strftime('%Y%m%d')
WORK_DIR = "/bbo_rietveld/hdd/work/" + user + STUDY_NAME + '/' + today
RESULT_DIR = "/bbo_rietveld/hdd/result/" + user + STUDY_NAME + '/' + today

In [14]:
file_list = glob.glob(target_dir + "/*.csv")
print("file =",len(file_list))
print(glob.glob(target_dir + "/*.csv"))

file = 10
['/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_210min_20240713_152450_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_270min_20240713_162904_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_90min_20240713_131029_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_180min_20240713_145106_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_300min_20240713_170105_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_150min_20240713_141458_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_240min_20240713_155714_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_30min_20240713_120606_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_60min_20240713_123918_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_120min_20240713_134

In [15]:
# make work directory
! mkdir -p $WORK_DIR
! mkdir -p $RESULT_DIR

In [ ]:
if user == "yotsumoto":
    analyzed = ['/bbo_rietveld/yotsumoto/nas/20240706/YY05_anatase_mill_5h_500rpm_20240704_130732_G01_S01.csv','/bbo_rietveld/yotsumoto/nas/20240706/YY05_anatase_mill_2h_500rpm_20240704_183632_G01_S01.csv']
    while True:
        file_list = glob.glob(target_dir + "/*.csv")
        unanalyzed = [f for f in file_list if f not in analyzed]
        if len(unanalyzed) > 0:
            unanalyzed.sort()
            target_file = unanalyzed[0]
            print("target: ",target_file)
#             tio2_bbor_run(target_file)
            tio2_brookite_bbor_run(target_file)
            analyzed.append(target_file)
            print("analyzed: ",analyzed)
        else:
            time.sleep(60)
elif user == "kawasaki":
    analyzed = []
    while True:
        file_list = glob.glob(target_dir + "/*.csv")
        unanalyzed = [f for f in file_list if f not in analyzed]
        if len(unanalyzed) > 0:
            unanalyzed.sort()
            target_file = unanalyzed[0]
            print("target: ",target_file)
            mechano_bbor_run(target_file)
            analyzed.append(target_file)
            print("analyzed: ",analyzed)
        else:
            time.sleep(60)

Finished!!
Best trial:186
Best Rwp:4.390879346398617
Time: 1691.42 sec  (0:28:11.423423)
analyzed:  ['/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_120min_20240713_134336_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_150min_20240713_141458_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_180min_20240713_145106_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_210min_20240713_152450_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_240min_20240713_155714_G01_S01.csv', '/bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_270min_20240713_162904_G01_S01.csv']
target:  /bbo_rietveld/kawasaki/nas/KKYN6_20N_120rpm_under300um_CsPbBr3_300min_20240713_170105_G01_S01.csv


[I 2024-07-14 20:13:38,694] Finished trial#4 resulted in value: 86.01638995090227. Current best value is 86.01638995090227 with parameters: {'Phase_parameters refine phase0': True, 'Background type': 'Q^2 power series', 'Number of coefficients': 10, 'Background refine': True, 'Instrument_parameters refine Zero': True, 'Sample_parameters refine Shift': True, 'Sample_parameters refine SurfRoughA': True, 'Sample_parameters refine SurfRoughB': True, 'Peakshape_parameters refine U': False, 'Peakshape_parameters refine V': True, 'Peakshape_parameters refine W': True, 'Peakshape_parameters refine X': False, 'Peakshape_parameters refine Y': True, 'Peakshape_parameters refine SH/L': False, 'Mustrain refine': False}.
[I 2024-07-14 20:13:54,992] Finished trial#8 resulted in value: 64.41141937852663. Current best value is 64.41141937852663 with parameters: {'Phase_parameters refine phase0': True, 'Background type': 'chebyschev', 'Number of coefficients': 1, 'Background refine': False, 'Instrument_